In [ ]:
!pip install schedule


In [ ]:
import smtplib
from email.mime.text import MIMEText
import schedule
import time 
import requests
import pandas as pd
from bs4 import BeautifulSoup

def sendMail(me, you, msg):
    smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp.login(me, 'kvarywmdfppfishz') 
    msg = MIMEText(msg)
    msg['Subject'] = '가산_Benn_올리브영 진행중인 이벤트'
    smtp.sendmail(me, you, msg.as_string())
    smtp.quit()   
    
def event_list():

    url = "https://www.oliveyoung.co.kr/store/main/getEventList.do"
    html = requests.get(url)
    total_info = BeautifulSoup(html.text, "html.parser")
    evt = total_info.find_all("div", {'class': 'event_tab_cont'})

    for i in evt:
        title_a, content_a, date_a = i.find_all('p', {'class': 'evt_tit'}), i.find_all('p', {'class': 'evt_desc'}), i.find_all('p', {'class': 'evt_date'})

    title, content, date = [], [], []
    
    for a,b,c in zip(title_a, content_a, date_a):
        title.append(a.text)
        content.append(b.text)
        date.append(c.text)
    
    dic_data = { '제목': title, 
                 '혜택': content,
                 '기간': date }
    
    df = pd.DataFrame(dic_data)
    return df

   

def job():
    new_event = event_list() 
    # new_event가 리스트일 때, str(new_event) 안써주면 encoding 에러!     
    sendMail('ghwo1119@gmail.com', 'stephan9404@naver.com', str(new_event))

schedule.every(10).seconds.do(job)    

while True:
    schedule.run_pending()
    time.sleep(1)